# Almost Exact simulation of the 3/2 model

##  1. Assumption

According to Baldeaux(2013), The dynamics of the stock price under the 3/2 model under the risk-neutral measure are given by

$$
 \frac{dS_t}{S_t} = rdt + \sqrt{V_t}\rho dW_t^1 + \sqrt{V_t}\sqrt{1-\rho^2}dW_t^2 \tag{1}
$$

$$
 \frac{dV_t}{V_t} = \kappa (\theta - V_t)dt + \epsilon \sqrt{V_t}dW_t^1
$$

which is equivalent to

$$
 dV_t = \kappa V_t (\theta - V_t)dt + \epsilon V_t^{3/2}dW_t^1 \tag{2}
$$

where $W_t^1$ and $W_t^2$ are independent Brownian motions. Regarding the parameters, $r$ represents the constant interest rate, $\rho$ the instantaneous correlation between the return on the stock and the variance process and $\epsilon$ governs the volatility of volatility.The speed of mean reversion is given by $\kappa V_t$ and $\theta$ denotes the long-run mean of the variance process.

## 2.  Mathematical basis

Defining $X_t = \frac{1}{V_t}$, we obtain

$$
dX_t = (\kappa + \epsilon^2 - \kappa\theta X_t)dt - \epsilon \sqrt{X_t}dW_t^1 \tag{3}
$$

Hence, using the process $X_t$, we obtain the following dynamics for the stock price, where $u > t$

$
S_u = S_t \exp\lbrace r(u-t) - 1/2 \int_t^u(X_s)^{-1}ds + \rho \int_t^u({\sqrt{X_s})^{-1}dW_s^1}\rbrace \exp \lbrace \sqrt{1-\rho^2} \int_t^u(\sqrt{X_s})^{-1} dW_s^2\rbrace \tag{4}
$


From Baldeaux(2013), study $\log(X_t)$

$
\int_t^u({\sqrt{X_s})^{-1}dW_s^1} = \frac{1}{\epsilon} (log(\frac{X_t}{X_u}) + (k + \frac{\epsilon^2}{2})\int_{t}^{u}\frac{ds}{X_s}-k\theta(u-t)) \tag{5}
$

Therefore, the only thing we need to know is the distribution of $X_t$ and $\int_{t}^{u}\frac{ds}{X_s}$ conditional on $X_t$

## 3. Numerical method 

Using Broadie-Kaya algorithm, we specify the simulation as 4 steps

### Step 1) Simulate $X_u|X_t$ using the noncentral $\chi^2$ distribution

$X_u$ is distributed as a noncentral $ \chi^2 $ distribution

$$
\frac{X_u{\rm exp}\lbrace \kappa \theta (u-t) \rbrace}{c(u-t)} \sim \chi^2(\delta, \alpha) \tag{6}
$$

where
$$
\delta = \frac{4(\kappa + \epsilon^2)}{\epsilon^2}, \quad \alpha = \frac{X_t}{c(u-t)}, \quad c(t) = \frac{\epsilon^2({\rm exp}\lbrace \kappa\theta u \rbrace - 1)}{4\kappa\theta}
$$

### Step 2)  Simulate $\int_t^u \frac{ds}{X_s}$ Given $X_u, X_t$

We first derive the characteristic function of $\int_u^t \frac{ds}{X_s}$, which is provided in Baldeaux(2013)

$$
E\left({\rm exp}\left\lbrace -a^* \int_0^t \frac{ds}{X_s} \ \bigg| \ X_t \right\rbrace \right) = \frac{I_{\sqrt{\nu^2+8a/\epsilon^2}}\left(-\frac{j\sqrt{X_tX_u}}{{\rm sinh}\left(j\Delta\right)}\right)}{I_{\nu}\left(-\frac{j\sqrt{X_tX_u}}{{\rm sinh}\left(j\Delta\right)}\right)}\tag{7}
$$

where $j=-\frac{2\kappa\theta}{\epsilon^2}$, $\Delta=\frac{u\epsilon^2}{4}-\frac{t\epsilon^2}{4}$, $v=\frac{n}{2}-1$.


Then we use the characteristic function to generate moment $M1$, $M2$. For simplicity, we define $\mu = M1$, $\sigma = \sqrt{log(\frac{M2}{M1^2})}$, and assume that $\int_t^u \frac{ds}{X_s}$ follows the lognorm distribution $\mathscr{L}(\mu, \sigma)$, then we can simulate $\int_t^u \frac{ds}{X_s}$. (which is different from Baldeaux(2013))

### Step 3) Recover $\int_t^u({\sqrt{X_s})^{-1}dW_s^1}$ from (5)

### Step 4) Simulate $S_u$ given $S_t$,   $\int_t^u({\sqrt{X_s})^{-1}dW_s^1}$ and $\int_u^t \frac{ds}{X_s}$ via 

$$
log(S_u) \sim N(log(S_t)+r(\mu-t)-\frac{1}{2}\int_t^u \frac{ds}{X_s}+\rho\int_t^u({\sqrt{X_s})^{-1}dW_s^1},  \sigma^2(t,n))\tag{8}
$$
where 
$$
\sigma^2(t, \mu) = (1-\rho^2)\int_t^u \frac{ds}{X_s}
$$

The option price is $C_{BS}(K,S_u, \frac{\sigma(t, \mu)}{\sqrt{u-t}})$

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import numpy as np
import pandas as pd 
import pyfeng.sv32_mc as svae

First, we assign all the parameters given, and run the AEMC method.

In [35]:
S0=100
Ks = np.array([95,100,105])
theta = 0.218
rho = -0.99
sigma = np.sqrt(0.06)
r = 0
T = 0.5
kappas = np.array([22.84, 18.32, 19.76, 20.48])
vovs = np.array([8.56, 8.56, 3.2, 3.2])

In [42]:
table_for_AEMC = pd.DataFrame(index = [0.95,1,1.05], columns = ['Case II', 'Case III', 'Case IV', 'Case V'])
table_for_AEMC_std = pd.DataFrame(index = [0.95,1,1.05], columns = ['Case II', 'Case III', 'Case IV', 'Case V'])
table_for_EMC = pd.DataFrame(np.array([[10.364,10.055,11.657,11.724],[7.386,7.042,8.926,8.999],[4.938,4.586,6.636,6.710]]),
                             index = [0.95,1,1.05], columns = ['Case II', 'Case III', 'Case IV', 'Case V'])

In [51]:
for i in range(4):
    svae.Sv32McAe2(intr=r, texp=T, sigma=sigma, rho=rho, theta=theta, mr=kappas[i], vov=vovs[i], path_num = 1000)
    output1 = aemc.get_price(strike=Ks,spot=S0, texp=T, cp=1)
    table_for_AEMC.iloc[:,i] = output1[0]
    table_for_AEMC_std.iloc[:,i] = output1[1]

Now, we check the prices from AEMC, comparing to EMC

In [52]:
table_for_AEMC

,Case II,Case III,Case IV,Case V
0.95,10.512208,11.464762,10.421826,10.749179
1.00,10.512208,11.464762,10.421826,10.749179
1.05,10.512208,11.464762,10.421826,10.749179


In [48]:
table_for_EMC

,Case II,Case III,Case IV,Case V
0.95,10.364,10.055,11.657,11.724
1.00,7.386,7.042,8.926,8.999
1.05,4.938,4.586,6.636,6.710
